## Artificial Neural Networks

In [10]:
from tensorflow import keras
from keras import layers
from keras.constraints import maxnorm
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation,  Flatten, Input

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
import time

%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import stats

# use seaborn plotting defaults
import seaborn as sns; sns.set()

import warnings
warnings.filterwarnings('ignore')


In [11]:
df = pd.read_csv('../FeatureEngineering/MetaData/data6_&_odds.csv')
df.dropna(inplace=True)

In [12]:
train_data = df.loc[(df.season <= 2013) & (df.season >= 2007)]
valid_data = df.loc[(df.season > 2013) & (df.season < 2016)]
test_data = df.loc[df.season >= 2016]
full_train_data = pd.concat([train_data, valid_data], axis=0)

X, y = train_data.drop(columns=['home_team_wins']), train_data.home_team_wins
valid_X, valid_y = valid_data.drop(columns=['home_team_wins']), valid_data.home_team_wins
test_X, test_y = test_data.drop(columns=['home_team_wins']), test_data.home_team_wins

# Split our data
X_train, y_train = train_data.drop(columns=["game_date_est","season","game_id","home_team","visitor_team","home_team_id","visitor_team_id","home_team_wins","conference","conference_visitor"]), train_data.home_team_wins
X_val, y_val = valid_data.drop(columns=["game_date_est","season","game_id","home_team","visitor_team","home_team_id","visitor_team_id","home_team_wins","conference","conference_visitor"]), valid_data.home_team_wins
X_test, y_test = test_data.drop(columns=["game_date_est","season","game_id","home_team","visitor_team","home_team_id","visitor_team_id","home_team_wins","conference","conference_visitor"]), test_data.home_team_wins


### Start making my first FeedForward ANN

#### Use the whole dataset

In [13]:
# define the keras model
model = Sequential()
model.add(Dense(10, activation='tanh'))
model.add(Dense(4, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
# compile the keras model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
# fit the keras model on the dataset
model.fit(X_train, y_train, epochs=100, batch_size=32)
# evaluate the keras model
_, accuracy = model.evaluate(X_train, y_train)
print('Accuracy: %.2f' % (accuracy*100))

Epoch 1/100
216/216 [==============================] - 1s 3ms/step - loss: 0.6764 - accuracy: 0.5964
Epoch 2/100
216/216 [==============================] - 0s 2ms/step - loss: 0.6490 - accuracy: 0.5965
Epoch 3/100
216/216 [==============================] - 0s 2ms/step - loss: 0.6322 - accuracy: 0.6407
Epoch 4/100
216/216 [==============================] - 0s 2ms/step - loss: 0.6289 - accuracy: 0.6262
Epoch 5/100
216/216 [==============================] - 0s 2ms/step - loss: 0.6322 - accuracy: 0.6222
Epoch 6/100
216/216 [==============================] - 0s 2ms/step - loss: 0.6161 - accuracy: 0.6475
Epoch 7/100
216/216 [==============================] - 0s 2ms/step - loss: 0.6210 - accuracy: 0.6723
Epoch 8/100
216/216 [==============================] - 0s 2ms/step - loss: 0.6253 - accuracy: 0.6772
Epoch 9/100
216/216 [==============================] - 0s 2ms/step - loss: 0.6222 - accuracy: 0.6133
Epoch 10/100
216/216 [==============================] - 0s 2ms/step - loss: 0.6134 - accura

In [14]:
start_time = time.time()
# val_score = model.score(X_val, y_val)

preds = model.predict(X_val)
preds = (preds > 0.7)

# test_score = model.score(X_val, y_val)

76/76 [==============================] - 0s 2ms/step


In [15]:
target_names = ['home_loss', 'home_win']

print("Συνολικός χρόνος fit και predict: %s seconds" % (time.time() - start_time))
print(classification_report(y_val, preds, target_names=target_names))


Συνολικός χρόνος fit και predict: 3.807657241821289 seconds
              precision    recall  f1-score   support

   home_loss       0.46      0.97      0.62      1011
    home_win       0.89      0.19      0.31      1409

    accuracy                           0.51      2420
   macro avg       0.67      0.58      0.47      2420
weighted avg       0.71      0.51      0.44      2420



#### Univariate Feature Selection Dataset

In [17]:
X_train_uni = train_data[['diff_curr_win_pct','diff_curr_away_record','odds_home','odds_away','elo_diff']]
y_train_uni = y_train

X_val_uni = valid_data[['diff_curr_win_pct','diff_curr_away_record','odds_home','odds_away','elo_diff']]
y_val_uni = y_val

In [30]:
# define the keras model
model = Sequential()
model.add(Dense(10, activation='tanh'))
model.add(Dense(4, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
# compile the keras model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
# fit the keras model on the dataset
model.fit(X_train_uni, y_train_uni, epochs=100, batch_size=32)
# evaluate the keras model
_, accuracy = model.evaluate(X_train_uni, y_train_uni)
print('Accuracy: %.2f' % (accuracy*100))

Epoch 1/100
216/216 [==============================] - 1s 2ms/step - loss: 0.6268 - accuracy: 0.6545
Epoch 2/100
216/216 [==============================] - 0s 2ms/step - loss: 0.6029 - accuracy: 0.6793
Epoch 3/100
216/216 [==============================] - 0s 2ms/step - loss: 0.5966 - accuracy: 0.6812
Epoch 4/100
216/216 [==============================] - 0s 2ms/step - loss: 0.5916 - accuracy: 0.6836
Epoch 5/100
216/216 [==============================] - 0s 2ms/step - loss: 0.5898 - accuracy: 0.6841
Epoch 6/100
216/216 [==============================] - 0s 2ms/step - loss: 0.5879 - accuracy: 0.6867
Epoch 7/100
216/216 [==============================] - 0s 2ms/step - loss: 0.5832 - accuracy: 0.6888
Epoch 8/100
216/216 [==============================] - 0s 2ms/step - loss: 0.5833 - accuracy: 0.6923
Epoch 9/100
216/216 [==============================] - 0s 2ms/step - loss: 0.5826 - accuracy: 0.6941
Epoch 10/100
216/216 [==============================] - 0s 2ms/step - loss: 0.5816 - accura

In [31]:
start_time = time.time()
# val_score = model.score(X_val, y_val)

preds = model.predict(X_val_uni)
preds = (preds > 0.7)

# test_score = model.score(X_val, y_val)

76/76 [==============================] - 0s 2ms/step


In [35]:
target_names = ['home_loss', 'home_win']

print("Συνολικός χρόνος fit και predict: %s seconds" % (time.time() - start_time))
print(classification_report(y_val_uni, preds, target_names=target_names))


Συνολικός χρόνος fit και predict: 188.77770972251892 seconds
              precision    recall  f1-score   support

   home_loss       0.53      0.86      0.66      1011
    home_win       0.82      0.45      0.58      1409

    accuracy                           0.62      2420
   macro avg       0.67      0.66      0.62      2420
weighted avg       0.70      0.62      0.61      2420



#### ExtraTreesClassifier Dataset

In [33]:
X_train_extra = train_data[['odds_home', 'odds_away', 'home_elo', 'visitor_elo', 'elo_diff',
                    'eff_diff', 'eff_visitor', 'top_player_diff', 'diff_win_pct_prev_season',
                    'diff_home_record_last_season', 'ROAD_RECORD_home',
                    'diff_road_record_last_season', 'diff_win_pct_7_last_games', 'W_PCT_home',
                    'W_PCT_away', 'W_PCT_prev_away', 'diff_curr_away_record', 'HOME_RECORD_home', 'diff_curr_home_record',
                    'diff_curr_win_pct']]

y_train_extra = y_train

X_val_extra = valid_data[['odds_home', 'odds_away', 'home_elo', 'visitor_elo', 'elo_diff',
                    'eff_diff', 'eff_visitor', 'top_player_diff', 'diff_win_pct_prev_season',
                    'diff_home_record_last_season', 'ROAD_RECORD_home',
                    'diff_road_record_last_season', 'diff_win_pct_7_last_games', 'W_PCT_home',
                    'W_PCT_away', 'W_PCT_prev_away', 'diff_curr_away_record', 'HOME_RECORD_home', 'diff_curr_home_record',
                    'diff_curr_win_pct']]

y_val_extra = y_val

In [39]:
# define the keras model
model = Sequential()
model.add(Dense(8, activation='tanh'))
model.add(Dense(2, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
# compile the keras model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
# fit the keras model on the dataset
model.fit(X_train_extra, y_train_extra, epochs=100, batch_size=32)
# evaluate the keras model
_, accuracy = model.evaluate(X_train_extra, y_train_extra)
print('Accuracy: %.2f' % (accuracy*100))

Epoch 1/100
216/216 [==============================] - 1s 2ms/step - loss: 0.7316 - accuracy: 0.5445
Epoch 2/100
216/216 [==============================] - 0s 2ms/step - loss: 0.6368 - accuracy: 0.5994
Epoch 3/100
216/216 [==============================] - 0s 2ms/step - loss: 0.6271 - accuracy: 0.6432
Epoch 4/100
216/216 [==============================] - 0s 2ms/step - loss: 0.6297 - accuracy: 0.6651
Epoch 5/100
216/216 [==============================] - 0s 2ms/step - loss: 0.6208 - accuracy: 0.6268
Epoch 6/100
216/216 [==============================] - 0s 2ms/step - loss: 0.6227 - accuracy: 0.6238
Epoch 7/100
216/216 [==============================] - 0s 2ms/step - loss: 0.6219 - accuracy: 0.5981
Epoch 8/100
216/216 [==============================] - 0s 2ms/step - loss: 0.6196 - accuracy: 0.6464
Epoch 9/100
216/216 [==============================] - 0s 2ms/step - loss: 0.6215 - accuracy: 0.6365
Epoch 10/100
216/216 [==============================] - 0s 2ms/step - loss: 0.6221 - accura

In [40]:
start_time = time.time()
# val_score = model.score(X_val, y_val)

preds = model.predict(X_val_extra)
preds = (preds > 0.7)

# test_score = model.score(X_val, y_val)

76/76 [==============================] - 0s 2ms/step


In [41]:
target_names = ['home_loss', 'home_win']

print("Συνολικός χρόνος fit και predict: %s seconds" % (time.time() - start_time))
print(classification_report(y_val_extra, preds, target_names=target_names))


Συνολικός χρόνος fit και predict: 2.04227876663208 seconds
              precision    recall  f1-score   support

   home_loss       0.58      0.72      0.64      1011
    home_win       0.76      0.62      0.68      1409

    accuracy                           0.66      2420
   macro avg       0.67      0.67      0.66      2420
weighted avg       0.68      0.66      0.67      2420



#### RFECV Dataset

In [53]:
X_train_rcv = train_data[['missing_player_diff','odds_home','odds_away','visitor_elo']]
y_train_rcv = y_train

X_val_rcv = valid_data[['missing_player_diff','odds_home','odds_away','visitor_elo']]
y_val_rcv = y_val

In [60]:
# define the keras model
model = Sequential()
model.add(Dense(8, activation='tanh'))
model.add(Dense(2, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
# compile the keras model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
# fit the keras model on the dataset
model.fit(X_train_rcv, y_train_rcv, epochs=100, batch_size=32)
# evaluate the keras model
_, accuracy = model.evaluate(X_train_rcv, y_train_rcv)
print('Accuracy: %.2f' % (accuracy*100))

Epoch 1/100
216/216 [==============================] - 1s 2ms/step - loss: 0.6756 - accuracy: 0.5965
Epoch 2/100
216/216 [==============================] - 1s 2ms/step - loss: 0.6746 - accuracy: 0.5965
Epoch 3/100
216/216 [==============================] - 1s 2ms/step - loss: 0.6745 - accuracy: 0.5965
Epoch 4/100
216/216 [==============================] - 1s 2ms/step - loss: 0.6746 - accuracy: 0.5965
Epoch 5/100
216/216 [==============================] - 0s 2ms/step - loss: 0.6745 - accuracy: 0.5965
Epoch 6/100
216/216 [==============================] - 0s 2ms/step - loss: 0.6745 - accuracy: 0.5965
Epoch 7/100
216/216 [==============================] - 0s 2ms/step - loss: 0.6745 - accuracy: 0.5965
Epoch 8/100
216/216 [==============================] - 0s 2ms/step - loss: 0.6746 - accuracy: 0.5965
Epoch 9/100
216/216 [==============================] - 0s 2ms/step - loss: 0.6746 - accuracy: 0.5965
Epoch 10/100
216/216 [==============================] - 0s 2ms/step - loss: 0.6745 - accura

In [61]:
start_time = time.time()
# val_score = model.score(X_val, y_val)

preds = model.predict(X_val_rcv)
preds = (preds > 0.7)

# test_score = model.score(X_val, y_val)

76/76 [==============================] - 0s 2ms/step


In [62]:
target_names = ['home_loss', 'home_win']

print("Συνολικός χρόνος fit και predict: %s seconds" % (time.time() - start_time))
print(classification_report(y_val_rcv, preds, target_names=target_names))


Συνολικός χρόνος fit και predict: 1.4701318740844727 seconds
              precision    recall  f1-score   support

   home_loss       0.42      1.00      0.59      1011
    home_win       0.00      0.00      0.00      1409

    accuracy                           0.42      2420
   macro avg       0.21      0.50      0.29      2420
weighted avg       0.17      0.42      0.25      2420



#### PCA

In [64]:
from sklearn.decomposition import PCA

n_components = 30
pca = PCA(n_components=n_components).fit(X_train)
X_train_pca = pca.transform(X_train)
X_val_pca = pca.transform(X_val)
X_test_pca = pca.transform(X_test)

In [69]:
# define the keras model
model = Sequential()
model.add(Dense(8, activation='tanh'))
model.add(Dense(4, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
# compile the keras model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
# fit the keras model on the dataset
model.fit(X_train_pca, y_train, epochs=100, batch_size=32)
# evaluate the keras model
_, accuracy = model.evaluate(X_train_pca, y_train)
print('Accuracy: %.2f' % (accuracy*100))

Epoch 1/100
216/216 [==============================] - 1s 2ms/step - loss: 0.7875 - accuracy: 0.5587
Epoch 2/100
216/216 [==============================] - 0s 2ms/step - loss: 0.6475 - accuracy: 0.6490
Epoch 3/100
216/216 [==============================] - 0s 2ms/step - loss: 0.6198 - accuracy: 0.6612
Epoch 4/100
216/216 [==============================] - 0s 2ms/step - loss: 0.6098 - accuracy: 0.6652
Epoch 5/100
216/216 [==============================] - 0s 2ms/step - loss: 0.6061 - accuracy: 0.6699
Epoch 6/100
216/216 [==============================] - 0s 2ms/step - loss: 0.6029 - accuracy: 0.6765
Epoch 7/100
216/216 [==============================] - 0s 2ms/step - loss: 0.6012 - accuracy: 0.6780
Epoch 8/100
216/216 [==============================] - 0s 2ms/step - loss: 0.5993 - accuracy: 0.6781
Epoch 9/100
216/216 [==============================] - 0s 2ms/step - loss: 0.5976 - accuracy: 0.6799
Epoch 10/100
216/216 [==============================] - 0s 2ms/step - loss: 0.5969 - accura

In [70]:
start_time = time.time()
# val_score = model.score(X_val, y_val)

preds = model.predict(X_val_pca)
preds = (preds > 0.7)

# test_score = model.score(X_val, y_val)

76/76 [==============================] - 0s 2ms/step


In [71]:
target_names = ['home_loss', 'home_win']

print("Συνολικός χρόνος fit και predict: %s seconds" % (time.time() - start_time))
print(classification_report(y_val, preds, target_names=target_names))


Συνολικός χρόνος fit και predict: 1.1963891983032227 seconds
              precision    recall  f1-score   support

   home_loss       0.53      0.80      0.64      1011
    home_win       0.78      0.50      0.61      1409

    accuracy                           0.62      2420
   macro avg       0.66      0.65      0.62      2420
weighted avg       0.68      0.62      0.62      2420



#### Lasso Dataset

In [72]:
X_train_lasso = train_data[['elo_diff', 'odds_away', 'odds_home', 'eff_diff', 'eff_visitor',
              'missing_players', 'top_players', 'FT_PCT_home_7g', 'diff_avg_reb_away',
              'home_elo']]
y_train_lasso = y_train

X_val_lasso = valid_data[['elo_diff', 'odds_away', 'odds_home', 'eff_diff', 'eff_visitor',
              'missing_players', 'top_players', 'FT_PCT_home_7g', 'diff_avg_reb_away',
              'home_elo']]
y_val_lasso = y_val

In [77]:
# define the keras model
model = Sequential()
model.add(Dense(8, activation='tanh'))
model.add(Dense(2, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
# compile the keras model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
# fit the keras model on the dataset
model.fit(X_train_lasso, y_train_lasso, epochs=100, batch_size=32)
# evaluate the keras model
_, accuracy = model.evaluate(X_train_lasso, y_train_lasso)
print('Accuracy: %.2f' % (accuracy*100))

Epoch 1/100
216/216 [==============================] - 1s 2ms/step - loss: 0.6875 - accuracy: 0.5957
Epoch 2/100
216/216 [==============================] - 0s 2ms/step - loss: 0.6801 - accuracy: 0.5965
Epoch 3/100
216/216 [==============================] - 1s 2ms/step - loss: 0.6763 - accuracy: 0.5965
Epoch 4/100
216/216 [==============================] - 0s 2ms/step - loss: 0.6751 - accuracy: 0.5965
Epoch 5/100
216/216 [==============================] - 1s 2ms/step - loss: 0.6746 - accuracy: 0.5965
Epoch 6/100
216/216 [==============================] - 0s 2ms/step - loss: 0.6745 - accuracy: 0.5965
Epoch 7/100
216/216 [==============================] - 0s 2ms/step - loss: 0.6745 - accuracy: 0.5965
Epoch 8/100
216/216 [==============================] - 1s 3ms/step - loss: 0.6744 - accuracy: 0.5965
Epoch 9/100
216/216 [==============================] - 1s 3ms/step - loss: 0.6744 - accuracy: 0.5965
Epoch 10/100
216/216 [==============================] - 0s 2ms/step - loss: 0.6744 - accura

In [78]:
start_time = time.time()
# val_score = model.score(X_val, y_val)

preds = model.predict(X_val_lasso)
preds = (preds > 0.7)

# test_score = model.score(X_val, y_val)

76/76 [==============================] - 0s 2ms/step


In [79]:
target_names = ['home_loss', 'home_win']

print("Συνολικός χρόνος fit και predict: %s seconds" % (time.time() - start_time))
print(classification_report(y_val_lasso, preds, target_names=target_names))


Συνολικός χρόνος fit και predict: 1.2518720626831055 seconds
              precision    recall  f1-score   support

   home_loss       0.55      0.77      0.64      1011
    home_win       0.77      0.55      0.64      1409

    accuracy                           0.64      2420
   macro avg       0.66      0.66      0.64      2420
weighted avg       0.68      0.64      0.64      2420



#### SFS Forward Dataset

In [80]:
X_train_for_sfs = train_data[['num_possible_outcomes', 'odds_home', 'odds_away', 'HOME_RECORD_home',
                    'W_PCT_away', 'W_PCT_prev_home', 'ROAD_RECORD_prev_home', 'W_PCT_prev_away',
                    'HOME_RECORD_prev_away', 'ROAD_RECORD_prev_away', 'WIN_PRCT_home_3g',
                    'FT_PCT_home_3g', 'FT_PCT_away_3g', 'FG3_PCT_away_3g', 'WIN_PRCT_home_7g',
                    'FT_PCT_away_7g', 'REB_away_7g', 'diff_avg_ast_home', 'diff_avg_ast_away',
                    'diff_avg_fg3_pct_home', 'diff_avg_fg_pct_away', 'diff_avg_reb_away',
                    'top_players', 'eff', 'eff_visitor', 'G_7days', 'back2back',
                    'HG_7days_VISITOR', 'AG_7days_VISITOR', 'G_7days_VISITOR',
                    'back2back_visitor', 'missing_players', 'missing_players_visitor',
                    'home_elo', 'elo_diff', 'missing_player_diff', 'eff_diff',
                    'Home_Last_5_Avg_FG3_PCT_home', 'Home_Last_5_Avg_FG3_PCT_away',
                    'Away_Last_5_Avg_FG3_PCT_home', 'Away_Last_5_Avg_FT_PCT_away',
                    'diff_fg_pct_last_3_games', 'diff_fg3_pct_last_7_games',
                    'diff_ft_pct_last_3_games', 'diff_ast_last_3_games',
                    'diff_ast_last_7_games', 'diff_win_pct_prev_season',
                    'diff_home_record_last_season', 'diff_road_record_last_season',
                    'diff_curr_win_pct']]

y_train_for_sfs = y_train

X_val_for_sfs = valid_data[['num_possible_outcomes', 'odds_home', 'odds_away', 'HOME_RECORD_home',
                    'W_PCT_away', 'W_PCT_prev_home', 'ROAD_RECORD_prev_home', 'W_PCT_prev_away',
                    'HOME_RECORD_prev_away', 'ROAD_RECORD_prev_away', 'WIN_PRCT_home_3g',
                    'FT_PCT_home_3g', 'FT_PCT_away_3g', 'FG3_PCT_away_3g', 'WIN_PRCT_home_7g',
                    'FT_PCT_away_7g', 'REB_away_7g', 'diff_avg_ast_home', 'diff_avg_ast_away',
                    'diff_avg_fg3_pct_home', 'diff_avg_fg_pct_away', 'diff_avg_reb_away',
                    'top_players', 'eff', 'eff_visitor', 'G_7days', 'back2back',
                    'HG_7days_VISITOR', 'AG_7days_VISITOR', 'G_7days_VISITOR',
                    'back2back_visitor', 'missing_players', 'missing_players_visitor',
                    'home_elo', 'elo_diff', 'missing_player_diff', 'eff_diff',
                    'Home_Last_5_Avg_FG3_PCT_home', 'Home_Last_5_Avg_FG3_PCT_away',
                    'Away_Last_5_Avg_FG3_PCT_home', 'Away_Last_5_Avg_FT_PCT_away',
                    'diff_fg_pct_last_3_games', 'diff_fg3_pct_last_7_games',
                    'diff_ft_pct_last_3_games', 'diff_ast_last_3_games',
                    'diff_ast_last_7_games', 'diff_win_pct_prev_season',
                    'diff_home_record_last_season', 'diff_road_record_last_season',
                    'diff_curr_win_pct']]

y_val_for_sfs = y_val


In [81]:
# define the keras model
model = Sequential()
model.add(Dense(8, activation='tanh'))
model.add(Dense(2, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
# compile the keras model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
# fit the keras model on the dataset
model.fit(X_train_for_sfs, y_train_for_sfs, epochs=100, batch_size=32)
# evaluate the keras model
_, accuracy = model.evaluate(X_train_for_sfs, y_train_for_sfs)
print('Accuracy: %.2f' % (accuracy*100))

Epoch 1/100
216/216 [==============================] - 1s 2ms/step - loss: 0.6906 - accuracy: 0.5835
Epoch 2/100
216/216 [==============================] - 0s 2ms/step - loss: 0.6640 - accuracy: 0.6413
Epoch 3/100
216/216 [==============================] - 0s 2ms/step - loss: 0.6551 - accuracy: 0.6522
Epoch 4/100
216/216 [==============================] - 1s 2ms/step - loss: 0.6508 - accuracy: 0.6478
Epoch 5/100
216/216 [==============================] - 0s 2ms/step - loss: 0.6545 - accuracy: 0.6352
Epoch 6/100
216/216 [==============================] - 0s 2ms/step - loss: 0.6549 - accuracy: 0.6341
Epoch 7/100
216/216 [==============================] - 0s 2ms/step - loss: 0.6392 - accuracy: 0.6680
Epoch 8/100
216/216 [==============================] - 0s 2ms/step - loss: 0.6296 - accuracy: 0.6788
Epoch 9/100
216/216 [==============================] - 0s 2ms/step - loss: 0.6274 - accuracy: 0.6716
Epoch 10/100
216/216 [==============================] - 0s 2ms/step - loss: 0.6268 - accura

In [82]:
start_time = time.time()
# val_score = model.score(X_val, y_val)

preds = model.predict(X_val_for_sfs)
preds = (preds > 0.7)

# test_score = model.score(X_val, y_val)

76/76 [==============================] - 0s 2ms/step


In [83]:
target_names = ['home_loss', 'home_win']

print("Συνολικός χρόνος fit και predict: %s seconds" % (time.time() - start_time))
print(classification_report(y_val_for_sfs, preds, target_names=target_names))


Συνολικός χρόνος fit και predict: 8.458064556121826 seconds
              precision    recall  f1-score   support

   home_loss       0.58      0.75      0.65      1011
    home_win       0.77      0.61      0.68      1409

    accuracy                           0.67      2420
   macro avg       0.67      0.68      0.67      2420
weighted avg       0.69      0.67      0.67      2420



#### SFS Backwards Dataset

In [84]:
X_train_back_sfs = train_data[['num_possible_outcomes', 'odds_home', 'odds_away', 'HOME_RECORD_home',
                     'W_PCT_away', 'W_PCT_prev_away', 'HOME_RECORD_prev_away',
                     'ROAD_RECORD_prev_away', 'FT_PCT_home_3g', 'FG3_PCT_home_3g', 'PTS_away_3g',
                     'FG_PCT_away_3g', 'FT_PCT_away_3g', 'FG3_PCT_away_3g', 'PTS_home_7g',
                     'FG_PCT_home_7g', 'AST_home_7g', 'AST_away_7g', 'REB_away_7g',
                     'diff_avg_pts_away', 'diff_avg_ast_home', 'diff_avg_ast_away',
                     'diff_avg_fg3_pct_home', 'top_players', 'top_players_visitor', 'eff_visitor',
                     'G_7days', 'back2back', 'HG_7days_VISITOR', 'AG_7days_VISITOR',
                     'G_7days_VISITOR', 'back2back_visitor', 'home_elo', 'elo_diff',
                     'missing_player_diff', 'eff_diff', 'Home_Last_5_Avg_AST_home',
                     'Home_Last_5_Avg_REB_home', 'Home_Last_5_Avg_REB_away',
                     'Home_Last_5_Avg_FG3_PCT_away', 'Away_Last_5_Avg_PTS_home',
                     'Away_Last_5_Avg_FG3_PCT_home', 'Away_Last_5_Avg_AST_home',
                     'Away_Last_5_Avg_FT_PCT_away', 'diff_fg3_pct_last_3_games',
                     'diff_fg3_pct_last_7_games', 'diff_ft_pct_last_3_games',
                     'diff_ast_last_7_games', 'diff_reb_last_3_games',
                     'diff_win_pct_3_last_games']]

y_train_back_sfs = y_train

X_val_back_sfs = valid_data[['num_possible_outcomes', 'odds_home', 'odds_away', 'HOME_RECORD_home',
                     'W_PCT_away', 'W_PCT_prev_away', 'HOME_RECORD_prev_away',
                     'ROAD_RECORD_prev_away', 'FT_PCT_home_3g', 'FG3_PCT_home_3g', 'PTS_away_3g',
                     'FG_PCT_away_3g', 'FT_PCT_away_3g', 'FG3_PCT_away_3g', 'PTS_home_7g',
                     'FG_PCT_home_7g', 'AST_home_7g', 'AST_away_7g', 'REB_away_7g',
                     'diff_avg_pts_away', 'diff_avg_ast_home', 'diff_avg_ast_away',
                     'diff_avg_fg3_pct_home', 'top_players', 'top_players_visitor', 'eff_visitor',
                     'G_7days', 'back2back', 'HG_7days_VISITOR', 'AG_7days_VISITOR',
                     'G_7days_VISITOR', 'back2back_visitor', 'home_elo', 'elo_diff',
                     'missing_player_diff', 'eff_diff', 'Home_Last_5_Avg_AST_home',
                     'Home_Last_5_Avg_REB_home', 'Home_Last_5_Avg_REB_away',
                     'Home_Last_5_Avg_FG3_PCT_away', 'Away_Last_5_Avg_PTS_home',
                     'Away_Last_5_Avg_FG3_PCT_home', 'Away_Last_5_Avg_AST_home',
                     'Away_Last_5_Avg_FT_PCT_away', 'diff_fg3_pct_last_3_games',
                     'diff_fg3_pct_last_7_games', 'diff_ft_pct_last_3_games',
                     'diff_ast_last_7_games', 'diff_reb_last_3_games',
                     'diff_win_pct_3_last_games']]

y_val_back_sfs = y_val

In [88]:
# define the keras model
model = Sequential()
model.add(Dense(10, activation='tanh'))
model.add(Dense(4, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
# compile the keras model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
# fit the keras model on the dataset
model.fit(X_train_back_sfs, y_train_back_sfs, epochs=100, batch_size=32)
# evaluate the keras model
_, accuracy = model.evaluate(X_train_back_sfs, y_train_back_sfs)
print('Accuracy: %.2f' % (accuracy*100))

Epoch 1/100
216/216 [==============================] - 1s 2ms/step - loss: 0.6555 - accuracy: 0.6255
Epoch 2/100
216/216 [==============================] - 0s 2ms/step - loss: 0.6261 - accuracy: 0.6771
Epoch 3/100
216/216 [==============================] - 0s 2ms/step - loss: 0.6222 - accuracy: 0.6700
Epoch 4/100
216/216 [==============================] - 1s 2ms/step - loss: 0.6174 - accuracy: 0.6581
Epoch 5/100
216/216 [==============================] - 0s 2ms/step - loss: 0.6288 - accuracy: 0.6719
Epoch 6/100
216/216 [==============================] - 1s 2ms/step - loss: 0.6210 - accuracy: 0.6777
Epoch 7/100
216/216 [==============================] - 0s 2ms/step - loss: 0.6188 - accuracy: 0.6636
Epoch 8/100
216/216 [==============================] - 0s 2ms/step - loss: 0.6175 - accuracy: 0.6626
Epoch 9/100
216/216 [==============================] - 0s 2ms/step - loss: 0.6170 - accuracy: 0.6445
Epoch 10/100
216/216 [==============================] - 0s 2ms/step - loss: 0.6166 - accura

In [89]:
start_time = time.time()
# val_score = model.score(X_val, y_val)

preds = model.predict(X_val_back_sfs)
preds = (preds > 0.7)

# test_score = model.score(X_val, y_val)

76/76 [==============================] - 0s 2ms/step


In [90]:
target_names = ['home_loss', 'home_win']

print("Συνολικός χρόνος fit και predict: %s seconds" % (time.time() - start_time))
print(classification_report(y_val_back_sfs, preds, target_names=target_names))


Συνολικός χρόνος fit και predict: 1.2320890426635742 seconds
              precision    recall  f1-score   support

   home_loss       0.67      0.56      0.61      1011
    home_win       0.72      0.80      0.76      1409

    accuracy                           0.70      2420
   macro avg       0.70      0.68      0.69      2420
weighted avg       0.70      0.70      0.70      2420

